## Visual Chatbot

In [1]:
from dotenv import load_dotenv
import os

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")
# print(API_KEY)

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

from flask import Flask, request, jsonify, render_template
app = Flask(__name__)

### [1] OCR 사용 비주얼 챗봇 만들기
####  pytesseract : 이미지에서 텍스트를 추출(OCR) 
 파이썬에서 Tesseract OCR(Optical Character Recognition) 엔진을 사용할 수 있도록 해주는 래퍼 라이브러리이다. <br>
 Tesseract는 이미지에서 텍스트를 추출하는 데 매우 유용하며, 다양한 언어를 지원한다. <br>
 이 라이브러리를 사용하면 이미지 파일(예: PNG, JPG 등)에서 텍스트를 쉽게 추출할 수 있다. 

In [ ]:
import cv2
import base64
import numpy as np
from io import BytesIO
from PIL import Image
import pytesseract   

# Windows 사용자의 경우 직접 Tesseract 실행 파일 경로 지정
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# ChatGPT API를 호출하여 응답을 생성
def chat_with_gpt(prompt, chat_history):
    messages = chat_history + [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    chat_history.append({"role": "assistant", "content": response.choices[0].message.content})
    return response.choices[0].message.content, chat_history

# Base64 인코딩된 이미지를 OpenCV 및 Tesseract OCR로 처리.
def process_image(image_data):
    image_bytes = base64.b64decode(image_data)
    image = Image.open(BytesIO(image_bytes))
    opencv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    
    # OCR 적용
    extracted_text = pytesseract.image_to_string(image, lang='eng+kor')
    print(extracted_text)
    return extracted_text.strip()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/ask', methods=['POST'])
def ask():
    data = request.json
    user_input = data.get("text", "")
    image_data = data.get("image", "")
    chat_history = data.get("history", [])
    
    if image_data:
        extracted_text = process_image(image_data)
        user_input += " " + extracted_text
    
    response_text, chat_history = chat_with_gpt(user_input, chat_history)
    return jsonify({"response": response_text, "history": chat_history})

@app.route('/clear', methods=['POST'])
def clear_chat():
    """채팅 기록 초기화."""
    return jsonify({"history": []})

@app.route('/upload_image', methods=['POST'])
def upload_image():
    """웹캠으로 촬영한 이미지를 처리."""
    data = request.json
    image_data = data.get("image", "")
    if image_data:
        extracted_text = process_image(image_data) + '\nOCR로 검출한 위 내용에 대하여 설명해줘'
        return jsonify({"extracted_text": extracted_text})
    return jsonify({"error": "No image data received"})

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


### [2] OpenAI 이미지 분석 활용하기 
: 카메라에서 얻은 이미지를 ChatGPT에 질의 시 이미지로 전달하여 응답

In [1]:
from dotenv import load_dotenv
import os

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")
# print(API_KEY)

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

from flask import Flask, request, jsonify, render_template
app = Flask(__name__)

In [2]:
import cv2
import base64
import numpy as np
from io import BytesIO
from PIL import Image
import pytesseract   

# Windows 사용자의 경우 직접 Tesseract 실행 파일 경로 지정
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


def chat_with_gpt(prompt, chat_history):
    """ChatGPT API를 호출하여 응답을 생성."""
    messages = chat_history + [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    chat_history.append({"role": "assistant", "content": response.choices[0].message.content})
    return response.choices[0].message.content, chat_history

# OpenAI API를 사용하여 이미지 설명 생성
def generate_caption(image_base64,chat_history):
    response = client.chat.completions.create(
      # model = "gpt-4o-mini",     
      model = "gpt-4o",     # 성능 더 좋음
      messages = [
        {
          "role": "user",
          "content": [
            {"type": "text", "text": "이미지의 내용에 대한여 자세히 설명해주세요"},
            {
              "type": "image_url",
              "image_url": {
                 "url": f"data:image/jpeg;base64,{image_base64}"
              },
            },
          ],
        }
      ],
      # max_tokens=100,
    )    
    return response.choices[0].message.content, chat_history


@app.route('/')
def index():
    return render_template('index_openai.html')

@app.route('/ask', methods=['POST'])
def ask():
    data = request.json
    user_input = data.get("text", "")
    # image_data = data.get("image", "")
    chat_history = data.get("history", [])
    # print('질문하기')
    # if image_data:
    #     extracted_text = process_image(image_data)
    #     user_input += " " + extracted_text
    
    response_text, chat_history = chat_with_gpt(user_input, chat_history)
    return jsonify({"response": response_text, "history": chat_history})

@app.route('/clear', methods=['POST'])
def clear_chat():
    """채팅 기록 초기화."""
    return jsonify({"history": []})

@app.route('/upload_image', methods=['POST'])
def upload_image():
    """웹캠으로 촬영한 이미지를 처리."""
    data = request.json
    image_data = data.get("image", "")
    chat_history = data.get("history", [])
    # print('사진찍기')
   
    if image_data:
        response_text, chat_history = generate_caption(image_data,chat_history)        
        return jsonify({"response": response_text,"history": chat_history})
    return jsonify({"error": "No image data received"})

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Mar/2025 22:04:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 22:04:10] "GET /static/camera_openai.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 22:04:16] "POST /upload_image HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 22:05:34] "POST /upload_image HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 22:05:46] "POST /clear HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 22:06:00] "POST /upload_image HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 22:06:43] "POST /ask HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 22:08:20] "POST /upload_image HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 22:08:54] "POST /upload_image HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 22:09:05] "POST /clear HTTP/1.1" 200 -
